# Submission Systems 
Submission system play an important role, if you want to develop your pygromos code. Many times, they are hidden in the Simulation_runner blocks. But maybe you want to develop something, where you need direct access on the submission system? 

This notebook will give you some examples, how you can use the submission systems.
Note that all submission systems are write in the same ways, such you can exchange them quickly.

In [1]:
from pygromos.hpc_queuing.submission_systems import local # this executes your code in your local session.
from pygromos.hpc_queuing.submission_systems import lsf # this module can be used to submit to the lsf-queue (e.g. on euler)
from pygromos.hpc_queuing.submission_systems import dummy # this is a dummy system, that only prints the commands

## Local Submission

This system executes the commands directly in your current session. This allows you to locally test or execute your code. Maybe if your process needs much more time, you want later to switch to a submission system for job-queueing.

In [2]:
sub_local = local.LOCAL()
sub_local.verbose = True

In [3]:
bash_command = "sleep 2; echo \"WUHA\"; sleep 2"

job_id = sub_local.submit_to_queue(bash_command)
job_id

Submission Command: 	 s l e e p   2 ;   e c h o   " W U H A " ;   s l e e p   2
STDOUT: 
		b'WUHA\n'
END



0

In [4]:
#This is a dummy function, to not break the code!
sub_local.get_jobs_from_queue("FUN")

Searching ID:  FUN


/cluster/home/bschroed/code/pygromos/pygromos/hpc_queuing/submission_systems/local.py:159: UserWarning: Queue search was called, but no queue present!
  warnings.warn("Queue search was called, but no queue present!")


[]

## LSF Submission

The Lsf submission system allows to submit jobs to the IBM LSF-Queueing system.

You can submit and kill jobs and arrays to the queue, as well as getting information from the queuing list.

In [5]:
#Construct system:
sub_lsf = lsf.LSF(nmpi=1, job_duration = "24:00", max_storage=100)
sub_lsf.verbose = True

sub_lsf._refresh_job_queue_list_all_s = 0 #you must wait at least 1s to update job_queue list

### Queue Checking:

In [6]:
sub_lsf.get_queued_jobs()
sub_lsf.job_queue_list

Skipping refresh of job list, as the last update is 0:00:00.005620s ago


,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173242100,173242100,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-39,Test1,2021-05-25 12:59:00
173242093,173242093,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-019-01,Test1,2021-05-25 12:59:00
173243842,173243842,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-026-44,Test1,2021-05-25 13:04:00
173244155,173244155,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-25,Test1,2021-05-25 13:05:00
173244148,173244148,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-002-35,Test1,2021-05-25 13:05:00
173252102,173252102,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-005-12,Test1,2021-05-25 13:27:00
173252106,173252106,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-005-12,Test1,2021-05-25 13:27:00
173252140,173252140,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-014-23,Test1,2021-05-25 13:28:00
173252953,173252953,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-32,Test1,2021-05-25 13:32:00
173253332,173253332,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-024-29,Test1,2021-05-25 13:33:00


### Submission:
here you can submit jobs to the queue as bash commands

In [7]:
bash_command = "sleep 5; echo \"WUHA\"; sleep 2"
job_name = "Test1"

job_id = sub_lsf.submit_to_queue(command=bash_command, jobName=job_name)

writing tmp-submission-file to:  ./job_Test1.sh
Submission Command: 	 bsub -JTest1 -W 24:00 -o Test1.out -n 1 -R rusage[mem=100] ./job_Test1.sh
process returned id: 173256460


In [8]:
#search for the just submitted job in the queue
sub_lsf.search_queue_for_jobid(job_id)

Skipping refresh of job list, as the last update is 0:00:00.420446s ago


,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME


In [9]:
sub_lsf.search_queue_for_jobname("Test1")

Skipping refresh of job list, as the last update is 0:00:00.432559s ago


,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173242100,173242100,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-39,Test1,2021-05-25 12:59:00
173242093,173242093,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-019-01,Test1,2021-05-25 12:59:00
173243842,173243842,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-026-44,Test1,2021-05-25 13:04:00
173244155,173244155,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-25,Test1,2021-05-25 13:05:00
173244148,173244148,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-002-35,Test1,2021-05-25 13:05:00
173252102,173252102,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-005-12,Test1,2021-05-25 13:27:00
173252106,173252106,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-005-12,Test1,2021-05-25 13:27:00
173252140,173252140,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-014-23,Test1,2021-05-25 13:28:00
173252953,173252953,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-32,Test1,2021-05-25 13:32:00
173253332,173253332,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-024-29,Test1,2021-05-25 13:33:00


### Submitting multiple jobs

In [10]:
bash_command = "sleep 2; echo \"WUHA\"; sleep 2"
job_ids = []
for test in range(3):
    job_name = "Test"+str(test)

    job_id = sub_lsf.submit_to_queue(command=bash_command, jobName=job_name)
    job_ids.append(job_id)

writing tmp-submission-file to:  ./job_Test0.sh
Submission Command: 	 bsub -JTest0 -W 24:00 -o Test0.out -n 1 -R rusage[mem=100] ./job_Test0.sh
process returned id: 173256464
writing tmp-submission-file to:  ./job_Test1.sh
Submission Command: 	 bsub -JTest1 -W 24:00 -o Test1.out -n 1 -R rusage[mem=100] ./job_Test1.sh
process returned id: 173256467
writing tmp-submission-file to:  ./job_Test2.sh
Submission Command: 	 bsub -JTest2 -W 24:00 -o Test2.out -n 1 -R rusage[mem=100] ./job_Test2.sh
process returned id: 173256470


In [11]:
sub_lsf.search_queue_for_jobname("Te", regex=True)

,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173256460,173256460,bschroed,PEND,normal.24h,eu-a6-011-24,-,Test1,2021-05-25 13:44:00
173256464,173256464,bschroed,PEND,normal.24h,eu-a6-011-24,-,Test0,2021-05-25 13:44:00
173256467,173256467,bschroed,PEND,normal.24h,eu-a6-011-24,-,Test1,2021-05-25 13:44:00
173256470,173256470,bschroed,PEND,normal.24h,eu-a6-011-24,-,Test2,2021-05-25 13:44:00
173242096,173242096,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-15,Test0,2021-05-25 12:59:00
173242100,173242100,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-39,Test1,2021-05-25 12:59:00
173242093,173242093,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-019-01,Test1,2021-05-25 12:59:00
173242105,173242105,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-41,Test2,2021-05-25 12:59:00
173243842,173243842,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-026-44,Test1,2021-05-25 13:04:00
173243845,173243845,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-44,Test2,2021-05-25 13:04:00


### Killing a jobs

Remove a job the job queue

In [12]:
sub_lsf.kill_jobs(job_ids=[job_id])

Stopping: 173256470


In [14]:
sub_lsf.search_queue_for_jobname("Te", regex=True)

,JOBID,USER,STAT,QUEUE,FROM_HOST,EXEC_HOST,JOB_NAME,SUBMIT_TIME
173242096,173242096,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-15,Test0,2021-05-25 12:59:00
173242100,173242100,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-39,Test1,2021-05-25 12:59:00
173242093,173242093,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-019-01,Test1,2021-05-25 12:59:00
173242105,173242105,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-41,Test2,2021-05-25 12:59:00
173243842,173243842,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-026-44,Test1,2021-05-25 13:04:00
173243845,173243845,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-44,Test2,2021-05-25 13:04:00
173243840,173243840,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-026-45,Test0,2021-05-25 13:04:00
173244151,173244151,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-013-10,Test0,2021-05-25 13:05:00
173244155,173244155,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-009-25,Test1,2021-05-25 13:05:00
173244158,173244158,bschroed,DONE,normal.24h,eu-a6-011-24,eu-ms-002-24,Test2,2021-05-25 13:05:00
